# Lattice Green function

In this notebook, we will first manipulate a bit the Green function on a lattice, 

$$ G(k, i\omega_n) $$
where $k$ is a vector in the Brillouin Zone.


In [ ]:
%matplotlib inline
from pytriqs.plot.mpl_interface import plt
import numpy as np
from math import cos, pi
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, Idx

In [ ]:
beta = 1./0.4
t = 1.0 

n_k = 64
n_w = 128

# A mesh on a Brillouin Zone

We first need a simple Bravais Lattice in 2 dimensions,  its corresponding Brillouin Zone and a mesh on it.

In [ ]:
# Two unit vectors in R3
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])

BZ = BrillouinZone(BL)



The Green function $$ G(k, i\omega_n) $$
is on a cartesian product mesh : $ (k \times i\omega_n) \rightarrow {\mathcal{C}}$

In [ ]:
# Construct the Green function

# A regular mesh on the BZ with 30 x 30 points
kmesh = MeshBrillouinZone(BZ, n_k = n_k)

# Frequency mesh
wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)

g = Gf(mesh = MeshProduct(kmesh, wmesh),target_shape = [])

print g, g.rank, g.data.shape


In [ ]:
iGamma = 0.01 * 1j

def eps(k):
    return -2 * t* (cos(k[0]) + cos(k[1]))

# NB : loop is a bit slow in python ...
for k in g.mesh[0]:
    for w in g.mesh[1]:
        g[k,w] = 1/(w - eps(k) + iGamma)
        

## Simple tests

In [ ]:
print g.data.reshape(30,30,256)[:,0,128].imag
k = np.linspace(0, 2*np.pi, kmesh.linear_dims[0]+1, endpoint=True)
eps2 = lambda kx, ky: g([kx,ky,0],0).imag
np.vectorize(eps2)(k,0)[0:30]

print g.data.reshape(30,30,256)[:,0,128].imag - np.vectorize(eps2)(k,0)[0:30]

In [ ]:
k = [0.0, 0., 0]
g(k, 8) 

In [ ]:
mf = lambda n : (2*n+1)*pi/beta*1j
for k in g.mesh[0]:
    assert abs(g(k.value, 0) - 1/(mf(0) - eps(k) + iGamma))< 1.e-14

# 1. For nearest-neighbor model, the Fermi surface is nested

## 1.a Definition of the non-interacting Green's function and of a function that plots momentum distribution curves for that Green's function


\begin{equation}
G^R(\mathbf{k},\omega)=\frac{1}{\omega+i\eta-\epsilon(\mathbf{k})}
\end{equation}

with, in units where lattice spacing $a=1$,
\begin{equation}
\epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y})
\end{equation}



## 1.b Plot of the momentum distribution curve at the Fermi level

The Fermi surface is nested. 

    What do we mean by that?
    What is the nesting vector?

In [ ]:
# NOW HERE we plot G(k, om)
# We show how to slice for fixed k, fixed omega
# and make a SIMPLE plot

# Fix FIX the evaluator of the GF to be periodic
# We use it on a grid on BZ ...

k = np.linspace(-np.pi, np.pi, kmesh.linear_dims[0]+1, endpoint=True)
kx, ky = np.meshgrid(k, k)

spectral = lambda kx, ky: g([kx,ky,0],0).imag

plt.figure()
plt.pcolor(kx, ky, np.vectorize(spectral)(kx,ky))

In [ ]:
def plot_mdc(Nk):
    """
    Plot the MDC for a grid with Nk x Nk points
    """
    basis = 2.*np.pi*np.eye(2)
    kgrid = np.linspace(0, 1, Nk, endpoint=False)
    k = np.meshgrid(*(2*(kgrid,)), indexing='ij')
    k = np.tensordot(np.transpose(basis), k, 1)
    k = np.rollaxis(k, 0, 3)
    G= Green_2D(0.05j, k)
    plt.imshow(-np.imag(G)/np.pi, cmap="jet",interpolation="nearest")
    plt.xticks([0,Nk/2, Nk],[r"0",r"$\pi$","$2\pi$",])    
    plt.yticks([0,Nk/2, Nk],[r"0",r"$\pi$","$2\pi$",])
    plt.xlabel(r"$k_x$")
    plt.ylabel(r"$k_y$")
    plt.title("Momentum distribution curve (MDC) at the Fermi level")



## Density

In [ ]:
d = lambda k: g[Idx(k,k,0),:].density().real
kr = range(15)

plt.plot(kr, np.vectorize(d)(kr), '-o')